<a href="https://colab.research.google.com/github/amilkh/cs230-fer/blob/master/fer2013.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
%tensorflow_version 2.x #!pip3 install tensorflow-gpu==2.0  #unnecessary on aws if using conda_tensorflow2_p36

UsageError: Line magic function `%tensorflow_version` not found.


In [28]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sklearn.metrics as sm

import tensorflow as tf
from tensorflow.keras.layers import *

%matplotlib inline

In [10]:
data = pd.read_csv('fer2013/fer2013.csv')

#print('Number of samples in the dataset: ', data.shape[0])
emotion_cat = {0:'Angry', 1:'Disgust', 2:'Fear', 3:'Happy', 4:'Sad', 5:'Surprise', 6:'Neutral'}

# See the target distribution (check for imbalance)
#target_counts = data['emotion'].value_counts().reset_index(drop=False)
#target_counts.columns = ['emotion', 'number_samples']
#target_counts['emotion'] = target_counts['emotion'].map(emotion_cat)
#target_counts

# Transform images from strings to lists of integers. TODO: use an array cast
data['pixels'] = data['pixels'].apply(lambda x: [int(pixel) for pixel in x.split()])

In [11]:
# Select randomly 10 images
#random_seed = 1
#data_sample = data.sample(10, random_state=random_seed)
#f, axarr = plt.subplots(2, 5, figsize=(20, 10))

#i, j = 0, 0
#for idx, row in data_sample.iterrows():
#    img = np.array(row['pixels']).reshape(48,48)
#    axarr[i,j].imshow(img, cmap='gray')
#    axarr[i,j].set_title(emotion_cat[row['emotion']])
#    if j==4:
#        i += 1
#        j = 0
#    else:
#        j += 1

In [14]:
data_train = data[data['Usage']=='Training']
size_train = data_train.shape[0]
#print('Number samples in the training dataset: ', size_train)

data_dev = data[data['Usage']=='PublicTest']
size_dev = data_dev.shape[0]
#print('Number samples in the development dataset: ', size_dev)

# Retrieve train input and target
X_train, y_train = data_train['pixels'].tolist(), data_train['emotion'].values
# Reshape images to 4D (num_samples, width, height, num_channels)
X_train = np.array(X_train, dtype='float32').reshape(-1,48,48,1)
# Normalize images with max (the maximum pixel intensity is 255)
X_train = X_train/255.0

# Retrieve dev input and target
X_dev, y_dev = data_dev['pixels'].tolist(), data_dev['emotion'].values
# Reshape images to 4D (num_samples, width, height, num_channels)
X_dev = np.array(X_dev, dtype='float32').reshape(-1,48,48,1)
# Normalize images with max
X_dev = X_dev/255.0

In [35]:
#Do it yourself model
model_diy = tf.keras.models.Sequential([
    InputLayer(input_shape=(48,48,1),name="input"),
    Conv2D(filters=32,kernel_size=3,activation='relu',padding='same',name="conv1"),
    Dropout(0.25), BatchNormalization(),
    Conv2D(filters=32,kernel_size=3,activation='relu',padding='same',name="conv2"),
    Dropout(0.25), BatchNormalization(),
    MaxPool2D(pool_size=(2,2),name="maxpool1"),
    Conv2D(filters=64,kernel_size=3,activation='relu',padding='same',name="conv3"),
    Dropout(0.25), BatchNormalization(),
    Conv2D(filters=64,kernel_size=3,activation='relu',padding='same',name="conv4"),
    Dropout(0.25), BatchNormalization(),
    Flatten(),
    Dense(1024,input_shape=(24*24*64,1),activation='relu',name='fc1'),
    Dense(7,input_shape=(1024,1),activation='softmax',name='fc-softmax')
])

print("Accuracy after training")
model_diy.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

Accuracy after training


In [38]:
model_diy.fit(X_train,y_train,batch_size=32,epochs=5)
y_hat_dev = np.argmax(model_diy.predict(X_dev), axis=1)

sm.accuracy_score(y_hat_dev, y_dev)

Train on 28709 samples
Epoch 1/5
28709/28709 [==============================] - 36s 1ms/sample - loss: 0.2595 - accuracy: 0.9128
Epoch 2/5
28709/28709 [==============================] - 36s 1ms/sample - loss: 0.2550 - accuracy: 0.9184
Epoch 3/5
28709/28709 [==============================] - 36s 1ms/sample - loss: 0.1949 - accuracy: 0.9386
Epoch 4/5
28709/28709 [==============================] - 36s 1ms/sample - loss: 0.1831 - accuracy: 0.9422
Epoch 5/5
28709/28709 [==============================] - 36s 1ms/sample - loss: 0.1590 - accuracy: 0.9495


0.539704653106715